In [1]:
import sqlite3
conn = sqlite3.connect('/Users/oj/Documents/twitter_db/sentiment.db')
cur = conn.cursor()

In [2]:
def procesed_text(cur, limit):
    rows = cur.execute("""
        select processedUrl_UserName_HashTag from second_run limit {limit};
        """.format(limit=limit))
    for row in rows:
        yield row[0]

def sentiment_value(cur, limit):
    rows = cur.execute("""
        select score from second_run limit {limit};
        """.format(limit=limit))
    for row in rows:
        yield row[0]

In [3]:
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfTransformer
from sklearn.naive_bayes import MultinomialNB
from sklearn.pipeline import Pipeline
text_clf = Pipeline([('vect', CountVectorizer()),
                     ('tfidf', TfidfTransformer()),
                     ('clf', MultinomialNB()),
])

In [4]:
row_count = 10000
data = list(procesed_text(cur, row_count))
target = list(sentiment_value(cur, row_count))
text_clf = text_clf.fit(data, target)

In [5]:
print(len([x for x in target if x == '0']))

5812


In [6]:
docs_new = ['God is love', 'OpenGL on the GPU is fast', 'i hate this', 'this is good', 'this was terrible']
predicted = text_clf.predict(docs_new)

for tweet, sentiment in zip(docs_new, predicted):
    print('%r => %s' % (tweet, sentiment))

'God is love' => 1
'OpenGL on the GPU is fast' => 0
'i hate this' => 0
'this is good' => 0
'this was terrible' => 0


In [7]:
import pickle
with open('web_classifier.p', 'wb') as f:
    pickle.dump(text_clf, f, protocol=2)

In [16]:
clf4 = pickle.load( open( "web_classifier.p", "rb" ) )

In [17]:
clf4.predict(['worst day ever'])

array(['0'], 
      dtype='<U1')

In [19]:
clf4.predict(['best good day ever'])

array(['1'], 
      dtype='<U1')

In [126]:
from sklearn.externals import joblib
joblib.dump(text_clf, 'web_classifier.pkl') 

['web_classifier.pkl',
 'web_classifier.pkl_01.npy',
 'web_classifier.pkl_02.npy',
 'web_classifier.pkl_03.npy',
 'web_classifier.pkl_04.npy',
 'web_classifier.pkl_05.npy',
 'web_classifier.pkl_06.npy',
 'web_classifier.pkl_07.npy']

In [109]:
clf3 = joblib.load('filename.pkl') 

In [110]:
clf3.predict(['worst day ever'])

array(['0'], 
      dtype='<U1')